In [2]:
import tensorflow as tf
import numpy as np

In [3]:
print(tf.__version__)

2.1.0


## 1. A brief summary of major changes to TensorFlow 1.x

- API Cleanup
- Eager execution
- No more globals
- Functions, not sessions

In [6]:
# In Tensorflow 2.0, eager execution is enabled by default.
f = tf.executing_eagerly()
print(f)

# However, this API might return False in the following use cases:
# https://tensorflow.google.cn/api_docs/python/tf/executing_eagerly
# e.g.
# - tf.compat.v1.enable_eager_execution()
# - @tf.function
@tf.function
def test():
    return tf.executing_eagerly()
f = test()
print(f.numpy())

True
False


## 2. First example

$a=(\vec{b}+c)\cdot(c+2)$

In [14]:
# create a symbolic tensor: constant
# use .numpy() to get the value holding by this tensor when eager execution is enabled
# 
const = tf.constant(2.0, name='const')
print(const)
print(const.numpy())

# t.eval() is a shortcut for calling tf.compat.v1.get_default_session().run(t):
# After the graph has been launched in a session, the value of the Tensor 
# can be computed by passing it to tf.Session.run.

tf.Tensor(2.0, shape=(), dtype=float32)
2.0


In [17]:
# auto convert between np.ndarray and tf.Tensor
b = np.arange(0, 10)[:, np.newaxis]

# symbolic tensor: variable
c = tf.Variable(1.0, dtype=tf.float32, name='c')

# operations
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a') # same with a = d * e

print(f'rank: {tf.rank(a).numpy()}\nshape: {a.shape}\ndtype: {a.dtype}\ndata: \n {a.numpy()}')

rank: 2
shape: (10, 1)
dtype: <dtype: 'float32'>
data: 
 [[ 3.]
 [ 6.]
 [ 9.]
 [12.]
 [15.]
 [18.]
 [21.]
 [24.]
 [27.]
 [30.]]


## 3. `tf.GradientTape`: to trace operations for computing gradients later

In [21]:
# By default, the resources held by a GradientTape are released 
# as soon as GradientTape.gradient() method is called. 
# To compute multiple gradients over the same computation, 
# create a persistent gradient tape by setting `persistent=True`
# 
x1 = tf.Variable([[2.0]])
x2 = tf.constant([[3.0]])
with tf.GradientTape(persistent=True) as tape: # persistent=True
    y1 = x1 * x1    
    tape.watch(x2) # though x2 is not a variable, it is watched explictly
    y2 = x1 * x2

# now compute gradients
dy1_x1 = tape.gradient(y1, x1)
dy2_x2 = tape.gradient(y2, x2)
dy1_x2 = tape.gradient(y1, x2)

print(dy1_x1, dy2_x2, dy1_x2)

tf.Tensor([[4.]], shape=(1, 1), dtype=float32) tf.Tensor([[2.]], shape=(1, 1), dtype=float32) None
